In [20]:
def read_split(path:str,size:float):
    data = pd.read_csv(path)
    X = data.drop(columns=['y'])
    y = data.y
    return train_test_split(X,y, test_size=size, random_state=123)

In [38]:
import pandas as pd
import os
import copy
import glob
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import pickle as pick

from joblib import parallel_backend
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from  xgboost import XGBClassifier as xgb
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import TomekLinks 
from sklearn.metrics import balanced_accuracy_score 
import sklearn.metrics as met

import scipy.stats as stats

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [22]:
thetas = np.linspace(0,1,101)

In [23]:
paths = glob.glob('./../data/*csv')
paths

['./../data/data11.csv',
 './../data/data12.csv',
 './../data/data14.csv',
 './../data/data15.csv',
 './../data/data16.csv',
 './../data/data19.csv',
 './../data/data2.csv',
 './../data/data20.csv',
 './../data/data21.csv',
 './../data/data3.csv',
 './../data/data4.csv',
 './../data/data5.csv',
 './../data/data6.csv',
 './../data/data7.csv',
 './../data/data8.csv']

In [24]:
models = []
models.append(('Tree',DecisionTreeClassifier()))
models.append(('RF',RandomForestClassifier()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('XGB',xgb()))

In [25]:
parameters = []
parameters.append(('Tree',{'min_samples_leaf':[20,30,40,50],
                           'min_impurity_decrease':[0,0.01,0.03,0.05],
                           'ccp_alpha':[0,0.01,0.03,0.05],
                           'random_state':[123]}))


parameters.append(('RF',{'n_estimators':[500],
                         'min_samples_leaf':[20,30,40],
                         'min_impurity_decrease':[0,0.03,0.05],
                         'ccp_alpha':[0,0.03,0.05],
                         'oob_score':[False, True],
                         'random_state':[123],
                         'n_jobs':[-1]}))
                         

parameters.append(('KNN',{'n_neighbors':[3,5,7,11],
                          'algorithm':['ball_tree','kd_tree'],
                          'p':[1,2],
                          'leaf_size':[20,30,40],
                          'n_jobs':[-1]}))

parameters.append(('XGB',{'learning_rate': [0.3, 0.5],
                          'min_child_weight': [1,3,5],
                          'subsample': [0.6,0.8,1],
                          'verbosity': [0],
                          'colsample_bytree': [0.8,1],
                          'n_estimators': [100,300],
                          'random_state':[123],
                          'n_jobs':[-1]}))
parameters = dict(parameters)

In [26]:
dict_models = {'Tree':{'t':[],'nt':[], 'size':[]},
              'RF':{'t':[],'nt':[],'size':[]},
              'KNN':{'t':[],'nt':[],'size':[]},
              'XGB':{'t':[],'nt':[],'size':[]}
              }

In [55]:
# dict_models1 = copy.deepcopy(dict_models);
# dict_models2 = copy.deepcopy(dict_models);
# dict_models3 = copy.deepcopy(dict_models);
dict_models4 = copy.deepcopy(dict_models);

In [40]:
for path in paths:
    X_train, X_test, y_train, y_test = read_split(path,0.2)
    ros = SMOTE(random_state=123)
    x_ros, y_ros = ros.fit_resample(X_train,y_train)

    results = []
    names = []
    print('***************************************************************************')
    for name, model in models:
        print(f'{name}**************')
        model_grid = HalvingGridSearchCV(model,
                                  parameters[name],
                                  cv = 5,
                                  verbose=True,
                                  scoring='balanced_accuracy')
        model_grid.fit(x_ros, y_ros)
        print(model_grid.best_params_)
        # print(model_grid.best_score_)
        model.parameters = model_grid.best_params_
        #cv_results = cross_val_score(model,x_ros, y_ros, cv=5,scoring='balanced_accuracy')
        #results.append(cv_results)
        #names.append(name)
        #msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
        #print(msg)
        model.fit(x_ros,y_ros)
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print('ros: ',b_a_score)
        dict_models1[name]['t'].append(b_a_score)
        dict_models1[name]['size'].append(len(y_test))
        model.fit(X_train,y_train)
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print('noRos:',b_a_score)
        dict_models1[name]['nt'].append(b_a_score)
        print('**************')


***************************************************************************
Tree**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 23
max_resources_: 646
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 23
Fitting 5 folds for each of 64 candidates, totalling 320 fits
----------
iter: 1
n_candidates: 22
n_resources: 69
Fitting 5 folds for each of 22 candidates, totalling 110 fits
----------
iter: 2
n_candidates: 8
n_resources: 207
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 621
Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0.05, 'min_samples_leaf': 20, 'random_state': 123}
ros:  0.9939759036144578
noRos: 0.9939759036144578
**************
RF**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 23
max_resources_: 646
aggressive_elimination: Fal

ros:  0.9932432432432432
noRos: 0.9932432432432432
**************
KNN**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 117
max_resources_: 3182
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 48
n_resources: 117
Fitting 5 folds for each of 48 candidates, totalling 240 fits
----------
iter: 1
n_candidates: 16
n_resources: 351
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 1053
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 3159
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'algorithm': 'kd_tree', 'leaf_size': 30, 'n_jobs': -1, 'n_neighbors': 3, 'p': 1}
ros:  0.9842226246865422
noRos: 0.9880886040679855
**************
XGB**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 117
max_resources_: 3182
aggressive_elimination: False
factor: 3
-

----------
iter: 1
n_candidates: 22
n_resources: 69
Fitting 5 folds for each of 22 candidates, totalling 110 fits
----------
iter: 2
n_candidates: 8
n_resources: 207
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 621
Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0.03, 'min_samples_leaf': 50, 'random_state': 123}
ros:  0.5251916757940854
noRos: 0.6730558598028478
**************
RF**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 23
max_resources_: 634
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 23
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------
iter: 1
n_candidates: 18
n_resources: 69
Fitting 5 folds for each of 18 candidates, totalling 90 fits
----------
iter: 2
n_candidates: 6
n_resources: 207
Fitting 5 folds for each of 6 candidates, totalling 30 fits
--

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

...

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 18
n_resources: 60
Fitting 5 folds for each of 18 candidates, totalling 90 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0.05, 'min_samples_leaf': 20, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 123}
ros:  0.597972972972973
noRos: 0.5
**************
KNN**************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 302
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 48
n_resources: 20
Fitting 5 folds for each of 48 candidates, totalling 240 fits
----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'algorithm': 'ball_tree', 'leaf_size': 30, 'n_jobs': -1, 'n_neighbors': 3,

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 2
n_candidates: 8
n_resources: 180
Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'colsample_bytree': 0.8, 'learning_rate': 0.3, 'min_child_weight': 3, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 123, 'subsample': 1, 'verbosity': 0}
ros:  0.847972972972973
noRos: 0.4864864864864865
**************


In [41]:
dict_models1

{'Tree': {'t': [0.9939759036144578,
   0.9428885669832072,
   0.9851978266926721,
   0.9958538184201902,
   1.0,
   0.5251916757940854,
   0.8653846153846154,
   1.0,
   0.743127147766323,
   0.9035714285714285,
   0.8722222222222222,
   0.7892156862745098,
   0.8890977443609023,
   0.5,
   0.8614864864864865],
  'nt': [0.9939759036144578,
   0.9304415472933096,
   0.9797297297297297,
   0.9954113405440839,
   1.0,
   0.6730558598028478,
   0.5961538461538461,
   0.9507276507276508,
   0.8281786941580755,
   0.7059523809523809,
   0.8722222222222222,
   0.9019607843137255,
   0.8439849624060151,
   0.53125,
   0.597972972972973],
  'size': [89,
   1095,
   462,
   1265,
   366,
   94,
   56,
   198,
   297,
   67,
   41,
   40,
   45,
   20,
   41]},
 'RF': {'t': [0.9939759036144578,
   0.968895863511991,
   0.9932432432432432,
   0.991265158964274,
   1.0,
   0.42771084337349397,
   0.8653846153846154,
   1.0,
   0.7396907216494846,
   0.9833333333333334,
   0.8583333333333334,
   0.9

In [42]:
DF1 = pd.DataFrame.from_dict(dict_models1, orient='index')
DF1

,t,nt,size
Tree,"[0.9939759036144578, 0.9428885669832072, 0.985...","[0.9939759036144578, 0.9304415472933096, 0.979...","[89, 1095, 462, 1265, 366, 94, 56, 198, 297, 6..."
RF,"[0.9939759036144578, 0.968895863511991, 0.9932...","[0.9939759036144578, 0.9417618970974837, 0.993...","[89, 1095, 462, 1265, 366, 94, 56, 198, 297, 6..."
KNN,"[0.9759036144578312, 0.9334057621117013, 0.984...","[0.9939759036144578, 0.8574091957723053, 0.988...","[89, 1095, 462, 1265, 366, 94, 56, 198, 297, 6..."
XGB,"[0.9939759036144578, 0.9709345994956811, 0.993...","[0.9939759036144578, 0.9461478620097645, 0.986...","[89, 1095, 462, 1265, 366, 94, 56, 198, 297, 6..."


In [43]:
DF1.to_csv('DF1.csv')

In [28]:
for path in paths:
    X_train, X_test, y_train, y_test = read_split(path,0.2)
    rus = TomekLinks()
    x_rus, y_rus = rus.fit_resample(X_train,y_train)

    results = []
    names = []
    print('***************************************************************************')
    for name, model in models:
        print(f'{name}**************')
        model_grid = HalvingGridSearchCV(model,
                                         parameters[name],
                                         cv = 5,
                                         verbose=True,
                                         scoring='balanced_accuracy')
        model_grid.fit(x_rus, y_rus)
        print(model_grid.best_params_)
        # print(model_grid.best_score_)
        model.parameters = model_grid.best_params_
#         cv_results = cross_val_score(model,x_rus, y_rus, cv=5,scoring='balanced_accuracy')
#         results.append(cv_results)
#         names.append(name)
#         msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
#         print(msg)
        model.fit(x_rus,y_rus)
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print(b_a_score)
        dict_models2[name]['t'].append(b_a_score)
        model.fit(X_train,y_train)
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print('noRus:',b_a_score)
        dict_models2[name]['nt'].append(b_a_score)
        print('******************')

***************************************************************************
Tree**************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 353
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 20
Fitting 5 folds for each of 64 candidates, totalling 320 fits
----------
iter: 1
n_candidates: 22
n_resources: 60
Fitting 5 folds for each of 22 candidates, totalling 110 fits
----------
iter: 2
n_candidates: 8
n_resources: 180
Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'ccp_alpha': 0.01, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
0.9939759036144578
noRus: 0.9939759036144578
******************
RF**************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 353
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 20
Fitting 5 folds for each of 54 candidates, totall

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 0.8 0.8 0.8 0.7 0.7 0.7 0.8 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.8
 0.8 0.8 0.8 0.8 0.7 0.7]
  warnings.warn(
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the train scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan  

...

----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits
{'colsample_bytree': 0.8, 'learning_rate': 0.3, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 123, 'subsample': 1, 'verbosity': 0}
0.59375
noRus: 0.59375
******************
***************************************************************************
Tree**************
n_iterations: 2
n_required_iterations: 4
n_possible_iterations: 2
min_resources_: 20
max_resources_: 162
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 20
Fitting 5 folds for each of 64 candidates, totalling 320 fits
----------
iter: 1
n_candidates: 22
n_resources: 60
Fitting 5 folds for each of 22 candidates, totalling 110 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0.03, 'min_samples_leaf': 40, 'random_state': 123}
0.5709459459459459
noRus: 0.47297297297297297
******************
RF**************
n_iterations: 2
n_required_iterations: 4
n

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_jobs': -1, 'n_neighbors': 7, 'p': 2}
0.4864864864864865
noRus: 0.4864864864864865
******************
XGB**************
n_iterations: 2
n_required_iterations: 4
n_possible_iterations: 2
min_resources_: 20
max_resources_: 162
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 72
n_resources: 20
Fitting 5 folds for each of 72 candidates, totalling 360 fits
----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'colsample_bytree': 0.8, 'learning_rate': 0.3, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 123, 'subsample': 1, 'verbosity': 0}
0.4864864864864865
noRus: 0.4864864864864865
******************


In [29]:
dict_models2

{'Tree': {'t': [0.9939759036144578,
   0.926873759321852,
   0.9797297297297297,
   0.9954113405440839,
   1.0,
   0.6522453450164294,
   0.5961538461538461,
   0.9480249480249481,
   0.8264604810996563,
   0.7773809523809523,
   0.8722222222222222,
   0.9019607843137255,
   0.8439849624060151,
   0.5625,
   0.5709459459459459],
  'nt': [0.9939759036144578,
   0.9304415472933096,
   0.9784410699359153,
   0.9958538184201902,
   1.0,
   0.6670317634173055,
   0.6057692307692308,
   0.9507276507276508,
   0.7448453608247423,
   0.6976190476190476,
   0.8722222222222222,
   0.9019607843137255,
   0.8439849624060151,
   0.5,
   0.47297297297297297],
  'size': []},
 'RF': {'t': [0.9939759036144578,
   0.9539004238424809,
   0.9932432432432432,
   0.9940839069157653,
   1.0,
   0.4939759036144578,
   0.7403846153846154,
   1.0,
   0.6649484536082474,
   0.7142857142857143,
   0.8861111111111111,
   0.9019607843137255,
   0.8439849624060151,
   0.5,
   0.5],
  'nt': [0.9939759036144578,
   0.

In [36]:
DF2 = pd.DataFrame.from_dict(dict_models2, orient='index')
DF2

,t,nt,size
Tree,"[0.9939759036144578, 0.926873759321852, 0.9797...","[0.9939759036144578, 0.9304415472933096, 0.978...",[]
RF,"[0.9939759036144578, 0.9539004238424809, 0.993...","[0.9939759036144578, 0.9461478620097645, 0.993...",[]
KNN,"[0.9879518072289157, 0.8558801437845378, 0.986...","[0.9939759036144578, 0.8574091957723053, 0.988...",[]
XGB,"[0.9939759036144578, 0.9544101078384033, 0.986...","[0.9939759036144578, 0.9461478620097645, 0.986...",[]


In [37]:
DF2.to_csv('DF2.csv',sep=';')

In [44]:
for path in paths:
    X_train, X_test, y_train, y_test = read_split(path,0.2)
    results = []
    names = []
    print('***************************************************************************')
    for name, model in models:
        print(f'{name}**************')
        model_grid = HalvingGridSearchCV(model,
                                         parameters[name],
                                         cv = 5,
                                         verbose=True,
                                         scoring='balanced_accuracy')
        model_grid.fit(X_train, y_train)
        probs = np.array(model_grid.predict_proba(X_train)[:,0])
        perfor=[]
        for theta in thetas:
            pred = 1*(probs>=theta)
            perfor.append(met.balanced_accuracy_score(y_train,pred))
        perfor=np.array(perfor,dtype=np.float32)
        list1 = np.array(perfor)
        k = 7
        pos = []
        for i in range(k):
            max_v = np.argmax(list1)
            x=list1[max_v]
            for j in range(1,4):
                if j>=0 and j<=100:
                    list1[max_v-j]=-1*(list1[max_v-j]==x)+list1[max_v-j]*(list1[max_v-j]!=x)
                    list1[max_v+j]=-1*(list1[max_v+j]==x)+list1[max_v+j]*(list1[max_v+j]!=x)
            pos.append(max_v)
            list1[max_v] = -1
        x=abs((thetas[pos]-0.5)*(thetas[pos]>=0.5))+thetas[pos]*(thetas[pos]<0.5)
        tOptim=pos[np.where(x==np.max(x))[0][0]]/100
        print(tOptim)
        print(model_grid.best_params_)
        model.parameters = model_grid.best_params_
#     cv_results = cross_val_score(model,X_train, y_train, cv=5,scoring='balanced_accuracy')
#     results.append(cv_results)
#     names.append(name)
#     msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
#     print(msg)
        model.fit(X_train,y_train)
        probs=np.array(model.predict_proba(X_test)[:,0])
        b_a_score = met.balanced_accuracy_score(y_test, model.predict(X_test))
        print('noTruncated:',b_a_score)
        dict_models3[name]['nt'].append(b_a_score)
        b_a_score = met.balanced_accuracy_score(y_test, 1*(probs>=tOptim))
        print('Truncated',b_a_score)
        dict_models3[name]['t'].append(b_a_score)
        print('******************')

Tree**************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 354
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 20
Fitting 5 folds for each of 64 candidates, totalling 320 fits
----------
iter: 1
n_candidates: 22
n_resources: 60
Fitting 5 folds for each of 22 candidates, totalling 110 fits
----------
iter: 2
n_candidates: 8
n_resources: 180
Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.77
{'ccp_alpha': 0.01, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
noTruncated: 0.9939759036144578
Truncated 0.4166666666666667
******************
RF**************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 354
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 20
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------
iter: 1
n_candidates: 18
n_reso

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 2
n_candidates: 8
n_resources: 180
Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.78333333 0.77333333 0

0.96
{'colsample_bytree': 0.8, 'learning_rate': 0.5, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 123, 'subsample': 0.8, 'verbosity': 0}
noTruncated: 0.9939759036144578
Truncated 0.5
******************
Tree**************
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 162
max_resources_: 4377
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 162
Fitting 5 folds for each of 64 candidates, totalling 320 fits
----------
iter: 1
n_candidates: 22
n_resources: 486
Fitting 5 folds for each of 22 candidates, totalling 110 fits
----------
iter: 2
n_candidates: 8
n_resources: 1458
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 4374
Fitting 5 folds for each of 3 candidates, totalling 15 fits
0.14
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
noTruncated: 0.9183030205483127
Truncated 0.08169697

IndexError: index 101 is out of bounds for axis 0 with size 101

In [48]:
max_v+j

101

In [56]:
dict_models3

{'Tree': {'t': [0.4166666666666667, 0.08169697945168732, 0.021558930064084703],
  'nt': [0.9939759036144578, 0.9183030205483127, 0.9784410699359153],
  'size': []},
 'RF': {'t': [0.012048192771084338, 0.12147915660711411],
  'nt': [0.9939759036144578, 0.9505338269220451],
  'size': []},
 'KNN': {'t': [0.5, 0.19318364719137293],
  'nt': [0.9939759036144578, 0.8574091957723053],
  'size': []},
 'XGB': {'t': [0.5, 0.14289929717259509],
  'nt': [0.9939759036144578, 0.9461478620097645],
  'size': []}}

In [ ]:
# pd.DataFrame.from_dict(dict_models, orient='index')

In [ ]:
for key,value in dict_models1.items():
    plt.plot(dict_models1[key]['t'],'o',label=key+'t')
    plt.plot(dict_models1[key]['nt'],'o',label=key+'nt')
    plt.legend()
    plt.show()

In [58]:
parameters = []
parameters.append(('Tree',{'class_weight':['balanced'],
                           'min_samples_leaf':[20,30,40,50],
                           'min_impurity_decrease':[0,0.01,0.03,0.05],
                           'ccp_alpha':[0,0.01,0.03,0.05],
                           'random_state':[123]}))

parameters.append(('RF',{'class_weight':['balanced'],
                         'n_estimators':[500],
                         'min_samples_leaf':[20,30,40],
                         'min_impurity_decrease':[0,0.03,0.05],
                         'ccp_alpha':[0,0.03,0.05],
                         'oob_score':[False, True],
                         'random_state':[123],
                         'n_jobs':[-1]}))

parameters.append(('XGB',{'sample_type':['weighted'],
                          'learning_rate': [0.3, 0.5],
                          'min_child_weight': [1,3,5],
                          'subsample': [0.6,1,0.8],
                          'verbosity' : [0],
                          'colsample_bytree': [0.8,1],
                          'n_estimators': [100,300],
                          'random_state':[123],
                          'n_jobs':[-1]}))
parameters = dict(parameters)

In [59]:
models1 = []
models1.append(('Tree',DecisionTreeClassifier()))
models1.append(('RF',RandomForestClassifier()))
models1.append(('XGB',xgb()))

In [60]:
for path in paths:
    X_train, X_test, y_train, y_test = read_split(path,0.2)
    results = []
    names = []
    print('***************************************************************************')
    for name, model in models1:
        model_grid = HalvingGridSearchCV(model,
                                         parameters[name],
                                         cv = 5,
                                         verbose=True,
                                         scoring='balanced_accuracy')
        model_grid.fit(X_train, y_train)
        model.parameters = model_grid.best_params_
    #cv_results = cross_val_score(model,X_train, y_train, cv=5,scoring='balanced_accuracy')
    #results.append(cv_results)
    #names.append(name)
    #msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
    #print(msg)
        model.fit(X_train,y_train)
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print('weighted: ',b_a_score)
        dict_models4[name]['t'].append(b_a_score)
        if name == 'XGB':
            model.parameters.pop('sample_type','weighted')
        else:
            model.parameters.pop('class_weight','balanced')
        model.fit(X_train,y_train)
    #pick.dump(model,open(f'./weighted_{name}.sav','wb'))
        b_a_score = balanced_accuracy_score(y_test, model.predict(X_test))
        print('no_weighted: ',b_a_score)
        dict_models4[name]['nt'].append(b_a_score)
        print('******************')

***************************************************************************
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 20
max_resources_: 354
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 64
n_resources: 20
Fitting 5 folds for each of 64 candidates, totalling 320 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 22
n_resources: 60
Fitting 5 folds for each of 22 candidates, totalling 110 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

...

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

----------
iter: 1
n_candidates: 22
n_resources: 60
Fitting 5 folds for each of 22 candidates, totalling 110 fits


/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/syseng/sdariza/.conda/envs/dataMining/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: Use

weighted:  0.597972972972973
no_weighted:  0.47297297297297297
******************
n_iterations: 2
n_required_iterations: 4
n_possible_iterations: 2
min_resources_: 20
max_resources_: 164
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 20
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------
iter: 1
n_candidates: 18
n_resources: 60
Fitting 5 folds for each of 18 candidates, totalling 90 fits
weighted:  0.5
no_weighted:  0.5
******************
n_iterations: 2
n_required_iterations: 4
n_possible_iterations: 2
min_resources_: 20
max_resources_: 164
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 72
n_resources: 20
Fitting 5 folds for each of 72 candidates, totalling 360 fits
----------
iter: 1
n_candidates: 24
n_resources: 60
Fitting 5 folds for each of 24 candidates, totalling 120 fits
weighted:  0.4864864864864865
no_weighted:  0.4864864864864865
******************


In [57]:
dict_models4

{'Tree': {'t': [], 'nt': [], 'size': []},
 'RF': {'t': [], 'nt': [], 'size': []},
 'KNN': {'t': [], 'nt': [], 'size': []},
 'XGB': {'t': [], 'nt': [], 'size': []}}

In [ ]:

x=[np.array(dict_models["Tree"]["t"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32),np.array(dict_models["Tree"]["nt"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32) ]
x=np.array(x)

In [ ]:
stats.chi2_contingency(x)[1]

In [35]:
np.array(dict_models["Tree"]["t"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32)

array([26.249998, 50.789997, 17.036001], dtype=float32)

In [ ]:
if min(stats.shapiro(DatosMAPETrain).pvalue,stats.shapiro(DatosMAPETest).pvalue)<0.05:
    pval=min(stats.mannwhitneyu(DatosMAPETrain,DatosMAPETest).pvalue,stats.fligner(DatosMAPETrain,DatosMAPETest).pvalue)
else:
    pval=min(stats.ttest_ind(DatosMAPETrain,DatosMAPETest).pvalue,stats.levene(DatosMAPETrain,DatosMAPETest).pvalue) 
DatosMAPE=np.concatenate((DatosMAPETrain, DatosMAPETest), axis=None)
if pval>=0.05:
  print("El modelo es consistente y tiene un MAPE de: ",DatosMAPE.mean())
pval=min(pval,sm.stats.acorr_ljungbox(DatosMAPE)[1][0])
if pval>=0.05:
  print("El modelo es válido para predecir")